# Advent of Code 2023 - Day 10: **Pipe Maze**

### Too weak today. Getting solution from https://github.com/ricbit/advent-of-code/blob/main/2023/adv17-r.py

In [9]:
import numpy as np
import heapq as hq
from queue import PriorityQueue
from dataclasses import dataclass
import time

In [10]:
import sys
import itertools

def shoelace(points):
  area = 0
  pairs = zip(points, itertools.islice(itertools.cycle(points), 1, None))
  for (y1, x1), (y2, x2) in pairs:
    area += x1 * y2 - x2 * y1
  return area / 2

DIRECTIONS = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}

def line_blocks():
  return [line.strip().split("\n") for line in sys.stdin.read().split("\n\n")]

class Table:
  def __init__(self, lines):
    self.table = lines
    self.w = len(self.table[0])
    self.h = len(self.table)

  @staticmethod
  def read():
    return Table([list(line.strip()) for line in sys.stdin])

  def iter_all(self, conditional=lambda x: True):
    for j, i in itertools.product(range(self.h), range(self.w)):
      if conditional(self.table[j][i]):
        yield j, i

  def valid(self, j, i):
    return 0 <= j < self.h and 0 <= i < self.w

  def cvalid(self, complex_pos):
    return 0 <= complex_pos.imag < self.h and 0 <= complex_pos.real < self.w

  def iter_neigh8(self, j, i, conditional=lambda x: True):
    for dj, di in itertools.product(range(-1, 2), repeat=2):
      if dj == 0 and di == 0:
        continue
      jj, ii = j + dj, i + di
      if self.valid(jj, ii) and conditional(self.table[jj][ii]):
        yield jj, ii

  def iter_neigh4(self, j, i, conditional=lambda x: True):
    for dj, di in [(0,1), (1,0), (0,-1), (-1,0)]:
      jj, ii = j + dj, i + di
      if self.valid(jj, ii) and conditional(self.table[jj][ii]):
        yield jj, ii

  def __getitem__(self, j):
    return self.table[j]

  def get(self, complex_position):
    return self.table[int(complex_position.imag)][int(complex_position.real)]

  def transpose(self):
    return Table(["".join(t) for t in zip(*self.table)])
  
  def clock90(self):
    return Table([list(reversed(col)) for col in zip(*self.table)])

  def copy(self):
    return Table([line.copy() for line in self.table])

In [11]:
with open('input.txt', 'r') as f:
    lines = [[int(c) for c in list(line)] for line in f.read().splitlines()]

In [12]:
import sys
import heapq
from collections import defaultdict

def heuristic(m, y, x, score):
  return m.h - y + m.w - x

def search(m, minsize, maxsize):
  visited = defaultdict(lambda: 1e6)
  vnext = [(heuristic(m, 0, 0, 0), 0, (0, 0), 2)]
  while vnext:
    old_heuristic, score, (y, x), direction = heapq.heappop(vnext)
    if old_heuristic > visited[((y, x), direction)]:
      continue
    if (y, x) == (m.h - 1, m.w - 1):
      return score
    for j, i in m.iter_neigh4(y, x):
      dj, di = j - y, i - x
      if (dj == 0 and direction == 0) or (di == 0 and direction == 1):
        continue
      dscore = 0
      for size in range(1, maxsize + 1):
        nj, ni = y + size * dj, x + size * di
        if not m.valid(nj, ni):
          break
        dscore += m[nj][ni]
        pure_score = score + dscore
        new_heuristic = heuristic(m, nj, ni, pure_score) + pure_score
        vec = ((nj, ni), 0 if dj == 0 else 1)
        if size >= minsize and new_heuristic < visited[vec]:
          heapq.heappush(vnext, (new_heuristic, pure_score) + vec)
          visited[vec] = new_heuristic

m = Table(lines)
print(search(m, 1, 3))
print(search(m, 4, 10))

1008
1210
